In [ ]:
from PIL import Image

def split_image(image_path, output_path, tile_size):
    image = Image.open(image_path)

    # Obter as dimensões da imagem
    width, height = image.size

    # Calcular o número de divisões horizontal e vertical
    num_tiles_horizontal = width // tile_size
    num_tiles_vertical = height // tile_size

    # Iterar sobre os tiles e cortar a imagem
    for i in range(num_tiles_horizontal):
        for j in range(num_tiles_vertical):
            # Calcular as coordenadas de início e fim do tile
            left = i * tile_size
            upper = j * tile_size
            right = left + tile_size
            lower = upper + tile_size

            # Cortar o tile da imagem original
            tile = image.crop((left, upper, right, lower))

            # Salvar o tile
            tile.save(f"{output_path}/tile_{i}_{j}.png")

# Test
image_path = "/content/1.png"
output_path = "output_tiles"
tile_size = 100  # Tamanho do tile (em pixels)

split_image(image_path, output_path, tile_size)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

def segment_image(model, image_path):
    image = Image.open(image_path).convert('RGB')  # Converter para imagem colorida
    preprocess = transforms.Compose([
        transforms.Resize((256, 256)),  # Redimensionar para o tamanho esperado pelo modelo
        transforms.ToTensor(),  # Converter para tensor
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalizar os valores dos pixels
    ])
    image = preprocess(image).unsqueeze(0)  # Adicionar a dimensão do lote (batch)

    # Executar a segmentação
    model.eval()  # Definir o modelo para o modo de avaliação
    with torch.no_grad():
        output = model(image)

    # Converter a saída para uma máscara binária
    output = torch.sigmoid(output)
    output = (output > 0.5).float()

    # Converter a máscara para um array numpy
    output = output.squeeze(0).squeeze(0).cpu().numpy()

    return output

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.dconv_down1 = DoubleConv(in_channels, 64)
        self.dconv_down2 = DoubleConv(64, 128)
        self.dconv_down3 = DoubleConv(128, 256)
        self.dconv_down4 = DoubleConv(256, 512)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dconv_up3 = DoubleConv(256 + 512, 256)
        self.dconv_up2 = DoubleConv(128 + 256, 128)
        self.dconv_up1 = DoubleConv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        out = self.conv_last(x)
        return out

model = UNet(in_channels=3, out_channels=1)  # Exemplo com 3 canais de entrada e 1 canal de saída
print(model)
image_path = '/content/output_tiles/tile_0_0.png'
segmented_mask = segment_image(model, image_path)
plt.imshow(segmented_mask, cmap='gray')
plt.axis('off')
plt.show()